**This file contains the code for building up the meta model(LGBM Model)**

Importing necessary files from drive

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from tqdm import tqdm
import sklearn

In [ ]:
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
from sklearn.model_selection import StratifiedKFold,KFold
import lightgbm as lgb
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_squared_error
import xgboost
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from scipy.stats import uniform,randint
from sklearn.model_selection import train_test_split

In [ ]:
import warnings
warnings.filterwarnings('ignore')

**Here the predictions from all the base models are concatenated which works as the input for my meta model.This prediction is done on the other 50% dataset.The test prediction is done on the intial 20% dataset which is the test set and after final training of my meta model it predicts on the predictions of the test set from the base model which can be used to evaluate the final preformance of my meta model.**

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/s1_pred_df.txt','rb')
s1_pred_df=pickle.load(file)

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/s1_test_df.txt','rb')
s1_test_df=pickle.load(file)

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/s2_predict_df.txt','rb')
s2_pred_df=pickle.load(file)

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/s2_predict_test_df.txt','rb')
s2_test_df=pickle.load(file)

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/s3_pred_df.txt','rb')
s3_pred_df=pickle.load(file)

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/s3_test_df.txt','rb')
s3_test_df=pickle.load(file)

In [ ]:
file=open('/content/drive/MyDrive/Project Energy Consumption/df_tr_red_final_modified.txt','rb')
df_tr_red_final=pickle.load(file)

In [ ]:
df_tr_red_final.reset_index(inplace=True)

In [ ]:
df_tr_red_final.drop(['index','timestamp'],axis=1,inplace=True)

In [ ]:
df_tr_red_final.drop('level_0',axis=1,inplace=True)

**Target Transformation**

1.   Here I am taking log1p of the meter readings and then I will evaluate my base models on RMSE which by default becomes the RMSLE(The evaluation metric on which we have to evaluate on).



In [ ]:
y_tr=np.log1p(df_tr_red_final['meter_reading'])
df_tr_red_final.drop('meter_reading',axis=1,inplace=True)

**Drpping the features which are not important**

In [ ]:
df_tr_red_final.drop(['cloud_coverage','sea_level_pressure','wind_direction','wind_speed',
                      'is_summer_month','is_pub_holiday'],axis=1,inplace=True)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(df_tr_red_final,y_tr,test_size=0.2,random_state=0)

In [ ]:
X_train_d1,X_train_d2,y_train_d1,y_train_d2=train_test_split(X_train,y_train,test_size=0.5,random_state=0)

**Concatenating the predicted data from the base model which serves as an input for my meta model**

In [ ]:
X_train_d2_pred=pd.concat([s1_pred_df,s2_pred_df,s3_pred_df],axis=1)

**Hyperparameter tuning of my meta model(LGBM Regressor)**

In [ ]:
params={'max_depth':[3,5,7,9,11],
'learning_rate':[0.1,0.01,0.03,0.05],
'colsample_bytree':[0.7,0.8,0.9,1.0],
'n_estimators':[300,500,800,1200],
'min_child_samples':[50,100,200,300,500]}


lgb_reg=LGBMRegressor()
random_lgb=RandomizedSearchCV(lgb_reg,params,n_iter=8,scoring='neg_root_mean_squared_error',cv=3,verbose=24,random_state=5,n_jobs=-1)
random_lgb.fit(X_train_d2_pred,y_train_d2)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  5.2min
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  9.9min
[Parallel(n_jobs=-1)]: Done   3 tasks      | elapsed: 10.3min
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 10.5min
[Parallel(n_jobs=-1)]: Done   6 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done   7 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed: 22.2min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 23.0min
[Parallel(n_jobs=-1)]: Done  11 tasks      | elapsed: 25.5min
[Parallel(n_jobs=-1)]: Done  12 tasks      | elapsed: 26.2min
[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed: 28.8min
[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed: 29.7min
[Parallel(n_jobs=-1)]: Done  15 tasks      | elapsed: 29

RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=LGBMRegressor(boosting_type='gbdt',
                                           class_weight=None,
                                           colsample_bytree=1.0,
                                           importance_type='split',
                                           learning_rate=0.1, max_depth=-1,
                                           min_child_samples=20,
                                           min_child_weight=0.001,
                                           min_split_gain=0.0, n_estimators=100,
                                           n_jobs=-1, num_leaves=31,
                                           objective=None, random_state=None,
                                           reg_alpha=0.0, reg_lambda=0.0,
                                           silen...
                   iid='deprecated', n_iter=8, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.7, 0.8, 0.9,
  

**Finding the best params from the random search**

In [ ]:
random_lgb.best_params_

{'colsample_bytree': 0.9,
 'learning_rate': 0.05,
 'max_depth': 7,
 'min_child_samples': 50,
 'n_estimators': 800}

In [ ]:
random_lgb.best_score_

-0.6108507543085028

**Fitting the model with the best params on the predictions from the base model and using the ground truth values from the other 50% data set.**

In [ ]:
meta_model=LGBMRegressor(n_estimators=800,min_child_samples=50,max_depth=7,learning_rate=0.05,colsample_bytree=0.9,n_jobs=-1)
meta_model.fit(X_train_d2_pred,y_train_d2)

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.9,
              importance_type='split', learning_rate=0.05, max_depth=7,
              min_child_samples=50, min_child_weight=0.001, min_split_gain=0.0,
              n_estimators=800, n_jobs=-1, num_leaves=31, objective=None,
              random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
              subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

**Using the predictions on the test set from the base models to evaluate my meta model**

In [ ]:
predictions=pd.concat([s1_test_df,s2_test_df,s3_test_df],axis=1)

In [ ]:
test_pred=meta_model.predict(predictions)

In [ ]:
np.sqrt(mean_squared_error(y_test,test_pred))

0.609958338958561

In [ ]:
filename='meta_model.txt'
my_file=open(filename,'wb')

In [ ]:
pickle.dump(meta_model,my_file)

                                                             **End of Notebook**